In [ ]:
!pip install kagglehub tensorflow opencv-python matplotlib

### Импорт модулей, подготовка датасета

In [ ]:
import os
import shutil
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os

if os.path.exists("my_dataset_final"):
    shutil.rmtree("my_dataset_final")


import kagglehub
print("Скачиваем датасет")
path = kagglehub.dataset_download("akhatova/pcb-defects")
print(f"Путь к файлам: {path}")


source_images = os.path.join(path, "PCB_DATASET", "images")
base_dir = "my_dataset_final"
target_classes = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']

print("Копируем папки")
for cls in target_classes:
    src = os.path.join(source_images, cls)
    dst = os.path.join(base_dir, cls)

    if os.path.exists(src):
        shutil.copytree(src, dst)
        print(f"   + {cls}: Готово")
    else:
        print(f"   - {cls}: Не найдено")


print("Результат")

IMG_SIZE = (96, 96)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

print(f"Классы для обучения: {train_ds.class_names}")


### Задаем гиперпараметры для данных

In [ ]:
data_dir = pathlib.Path('my_dataset_final')

IMG_SIZE = (96, 96)
BATCH_SIZE = 32


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE
)


class_names = train_ds.class_names

train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Архитектура и создание модели

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(96, 96, 3)),

  # Аугментация
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),

  # Свертки
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Dense(len(class_names))
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Обучение

In [ ]:
EPOCHS = 15
print(f"\n>>> Старт обучения на {EPOCHS} эпох...")
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)

### Квантование

In [ ]:
# Функция калибровки
def representative_data_gen():
  for input_value, _ in train_ds.take(100):
    yield [tf.cast(input_value, tf.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Важные параметры для ESP32
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

### Скачиваем файл

In [ ]:
def hex_to_c_array(hex_data, var_name):
    c_str = f'const unsigned char {var_name}[] = {{'
    c_str += ', '.join([f'0x{hex_data[i:i+2]}' for i in range(0, len(hex_data), 2)])
    c_str += '};\n'
    c_str += f'const unsigned int {var_name}_len = {len(hex_data) // 2};\n'
    return c_str

with open('model_data.h', 'w') as f:
    f.write(hex_to_c_array(tflite_model.hex(), 'model_data'))

print("\n🎉 ГОТОВО! Файл model_data.h создан. Скачай его.")